In [ ]:
pip install langchain faiss-cpu transformers sentence-transformers pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 455.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 20.6 MB/s eta 0:00:00


### a) Procesamiento del Documento
Extraer texto del documento PDF.

In [ ]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Ruta al documento Food Code 2022
pdf_path = "Food_Code_2022.pdf"
document_text = extract_text_from_pdf(pdf_path)

### b) División del Texto en Fragmentos
Dividir el texto en fragmentos más pequeños para indexación.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Dividir el texto en fragmentos (e.g., 1000 caracteres con 200 de superposición)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(document_text)

### c) Creación del Índice
Construir un índice utilizando FAISS para la recuperación de fragmentos relevantes.

In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Inicializar el embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Construir el índice FAISS
vector_store = FAISS.from_texts(chunks, embeddings)

# Guardar el índice
vector_store.save_local("food_code_faiss_index")

### d) Búsqueda de Fragmentos Relevantes
Recuperar fragmentos relevantes para una consulta dada.

In [ ]:
def retrieve_relevant_chunks(query, vector_store, top_k=5):
    return vector_store.similarity_search(query, k=top_k)

# Consulta
query = "What are the requirements for food temperature control?"
relevant_chunks = retrieve_relevant_chunks(query, vector_store)
for i, chunk in enumerate(relevant_chunks):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")

Chunk 1:
January 18, 2023 Version
Food Code
2022 Recommendations of the
United States Public Health Service
Food and Drug Administration
The Food Code is a model for safeguarding public health and ensuring food is unadulterated
and honestly presented when offered to the consumer. It represents FDA's best advice for a
uniform system of provisions that address the safety and protection of food offered at retail
and in food service.
This model is offered for adoption by local, state, and federal governmental jurisdictions for
administration by the various departments, agencies, bureaus, divisions, and other units
within each jurisdiction that have been delegated compliance responsibilities for food
service, retail food stores, or food vending operations. Alternatives that offer an equivalent
level of public health protection to ensure that food at retail and foodservice is safe are
recognized in this model.
iPrevious Editions of Codes
Recommended by The
United States Public Health Service

### e) Generación de Respuestas
Usar un modelo generativo para responder preguntas basándonos en los fragmentos recuperados.

In [ ]:
from transformers import pipeline

# Modelo de generación
qa_pipeline = pipeline("text2text-generation", model="t5-small")

# Construir el contexto y generar respuesta
context = " ".join([chunk.page_content for chunk in relevant_chunks])
prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
response = qa_pipeline(prompt, max_length=200, num_return_sequences=1)
print("Generated Answer:", response[0]["generated_text"])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (379334 > 512). Running this sequence through the model will result in indexing errors
